In [1]:
#| default_exp classes.DomoCard

In [5]:
#| export
from dataclasses import dataclass, field

from domolibrary.routes.card import card_routes

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.utils.DictDot as util_dd

ModuleNotFoundError: No module named 'domolibrary.routes.card'

Bad pipe message: %s [b'6\xce\x8c\xe6\x10\x94\xe0hH\xa3\x11f\xb5L\x7f\x1cf\xa0 \xd8\x9a\x00L\xcbS\xec\xfb!$\xc1\xfd\xf5\xe6\xe7/\xe35\xe8v\x1c\xa1,\x1d\x11\xae\xea\xbb\x10\xc0h\x00\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00', b'\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18']
Bad pipe message: %s [b'\xa5\x04\x0cQm\x90\xc4\x17\x85']
Bad pipe message: %s [b"=\xe8\xbd\x8d\xa0\xb1\xfc\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x0

In [ ]:
#| export
@dataclass
class DomoCard(Base):
    id: str
    full_auth: DomoFullAuth = field(repr=False)
    title: str = None
    description: str = None
    type: str = None
    urn: str = None
    chart_type: str = None
    dataset_id: str = None
    certification: field(default_factory=dict) = None
    owner_members: field(default_factory=list) = None

    def __post_init__(self):
        Base().__init__()

        self.domo_instance = self.full_auth.domo_instance
        # self.Definition = CardDefinition(self)

    def display_url(self) -> str:
        return f'https://{self.domo_instance}.domo.com/kpis/details/{self.id}'

    @classmethod
    async def get_from_id(cls, id: str, full_auth: DomoFullAuth, debug: bool = False):
        res = await card_routes.get_card_metadata(full_auth=full_auth, card_id=id, debug=debug)

        if res.status == 200:
            dd = DictDot(res.response[0])

            card = cls(
                full_auth=full_auth,
                id=dd.id,
                title=dd.title,
                description=dd.description,
                type=dd.type,
                urn=dd.urn,
                certification=dd.certification,
                owner_members=dd.owners,
                chart_type=dd.metadata.chartType
            )

            if dd.datasources:
                card.dataset_id = dd.datasources[0].dataSourceId

            return card